# TAREA 4
## Importar librerías y módulos de Python
##### Importamos las librerías/módulos específicos de la siguiente forma.

In [ ]:
# Importacion.
# import librosa
from scipy.io import wavfile
import IPython
import os
import numpy as np
import matplotlib.pyplot as plt

## Especificar directorios de entrada y salida
##### Aquí definimos los directorios donde guardaremos los audios con los que vamos a trabajar, así como dónde se van a guardar aquellos que generamos a lo largo de la práctica.

In [ ]:
# Directorios que usaremos.
cwd = os.getcwd()
audio_input_path = os.path.join(cwd, os.path.join('audio', 'examples'))  # cambiar '_input' por 'examples'
audio_output_path = os.path.join(cwd, os.path.join('audio', '_output'))
print(f'Directorio con los audios de entrada: {audio_input_path}')
print(f'Directorio donde guardaremos los audios generados: {audio_output_path}\n')

## Cargar el archivo de audio
##### Cargamos el archivo de audio .wav en este caso.

In [ ]:
# Cargamos el archivo de audio.
filename = os.path.join(audio_input_path, 'the_last_of_us_reduced.wav')
# audio_data, sample_rate = librosa.load(filename, sr=None, mono=False)
sample_rate, audio_data = wavfile.read(filename)
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')

##### Vamos a escucharlo. Para que esto se haga correctamente, hay que indicarle la frecuencia de muestreo.

In [ ]:
IPython.display.Audio(audio_data.T, rate=sample_rate) # .T se pasa únicamente si es audio estéreo.

## Conversión de estereo a mono
##### Ahora, por simplificación, vamos a calcular la media por canal para obtener un sonido mono.

In [ ]:
# Convertimos a mono mediante la media por canal (simplificacion).
new_data_mono = audio_data.mean(axis=1)  # Column-wise.
# Mantenemos la misma resolucion que antes.
new_data_mono = new_data_mono.astype(np.int16)

tamaño_bytes = new_data_mono.nbytes  # Tamaño en bytes del array
tamaño_mb = tamaño_bytes / (1024 * 1024)
print('Nuevos datos de audio (mono):')
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')
print(f'Número de canales: {new_data_mono[:5]}...')
print(f"El tamaño del audio es: {tamaño_mb:.2f} MB")

##### Vamos a guardarlo

In [ ]:
# Guardamos el archivo mono a un fichero de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path, 'the_last_of_us_reduced.wav'),
    rate=sample_rate,
    data=new_data_mono
)

##### Vamos a escucharlo de nuevo

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate)

## Mostrar una gráfica en el tiempo para ambos audios

In [ ]:
ampl_estereo = len(audio_data)
ampl_mono = len(new_data_mono)
print(f'Número de muestras del audio con fs=48 kHz (valores de amplitud): {ampl_estereo}')
print(f'Número de muestras del audio con fs=24 kHz (valores de amplitud): {ampl_mono}')

In [ ]:
# Construimos el array para el eje x que representa el tiempo de la grabación.
# Tiene la forma: np.arange(Vi, Vf, P).
t1 = np.arange(0, ampl_estereo/sample_rate, 1/sample_rate)
t2 = np.arange(0, ampl_mono/sample_rate, 1/sample_rate)

In [ ]:
print(t1)
print(t2)

In [ ]:
# Creamos la figura.
fig, ax = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Solo mostramos las primeras 50 muestras de amplitud (por claridad).
end = 50

# Señal a 44 kHz.
ax[0].plot(t1[:end], audio_data[:end], marker='X')
ax[0].set_title(f'Audio en el dominio del tiempo muestreado a {sample_rate} Hz')
ax[0].set_ylabel('Amplitud')
ax[0].grid(True)

# Señal a 44 kHz.
# Utilizamos ratio para ajustar el eje x de ambas gráficas
# ya que la fs es menor en esta señal.
ratio = sample_rate / sample_rate  
ax[1].plot(t2[:int(end/ratio)], new_data_mono[:int(end/ratio)], c='tab:red', marker='X')
ax[1].set_title(f'Audio en el dominio del tiempo muestreado a {sample_rate} Hz')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Amplitud')
ax[1].grid(True)

# Mostramos la figura.
plt.tight_layout()
plt.show()


## Explicación
### Ahora se va a explicar que es recuencia de muestreo, aliasing, profundidad de bits, ancho de banda y tasa de bits.

##### - Frecuencia de muestreo: Es el número de muestras que se toman por segundo de una señal analógica para convertirla en digital. Una mayor frecuencia captura más detalles del sonido.

##### - Aliasing: Es un error que ocurre cuando la frecuencia de muestreo es demasiado baja y las altas frecuencias se "confunden" con frecuencias más bajas, generando distorsión en la señal digital.

##### - Profundidad de bits: Es la cantidad de bits usados para representar cada muestra. Mayor profundidad permite una representación más precisa del audio, ofreciendo un mayor rango dinámico y menor ruido.

##### - Ancho de banda: Es el rango de frecuencias que una señal puede transmitir o reproducir. En audio, define la cantidad de información sonora que se puede captar y reproducir.

##### - Tasa de bits: Es la cantidad de datos (en bits) procesados o transmitidos por segundo. Se relaciona con la calidad del audio y se calcula multiplicando la frecuencia de muestreo, la profundidad de bits y el número de canales.

## Transformada de Fourier para el audio mono

In [ ]:
# La longitud del array de datos y el
# sample rate (frecuencia de muestreo).
n = len(new_data_mono)
Fs = sample_rate

# Working with mono audio, there are one channel in the audio data.
# Let's retrieve each channel seperately:
# ch1 = np.array([data[i][0] for i in range(n)]) #channel 1
# We can then perform a Fourier analysis on the first
# channel to see what the spectrum looks like.

# Calculando la Transformada Rapida de Fourier (FFT) en audio mono.
ch_Fourier = np.fft.fft(new_data_mono)  # ch1

# Solo miramos frecuencia por debajo de Fs/2
# (Nyquist-Shannon) --> Spectrum.
abs_ch_Fourier = np.absolute(ch_Fourier[:n//2])

# Graficamos.
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud', labelpad=10)
plt.xlabel('$f$ (Hz)', labelpad=10)
plt.show()

### Por qué se aplica la FFT y qué muestra la gráfica
##### Transformada Rápida de Fourier (FFT): Permite pasar la señal del dominio del tiempo (amplitud vs. muestras/tiempo) al dominio de la frecuencia (amplitud vs. frecuencias).
##### Dominio de la frecuencia: Muestra en qué frecuencias está concentrada la energía de la señal. Los picos indican las componentes de mayor amplitud (por ejemplo, graves, medios o agudos).
##### Por qué la mitad del espectro: Para señales de audio mono (reales), la FFT es simétrica en torno a la frecuencia 0, por lo que basta graficar hasta sample_rate/2 (Nyquist).
##### Interpretación: Con esta gráfica se puede identificar las zonas donde el audio tiene más potencia (por ejemplo, un pico fuerte en frecuencias bajas si hay mucho contenido grave). En música o voz, gran parte de la energía se concentra en rangos de frecuencias específicas.

## Energia del espectrograma y frecuencia de corte
##### Ahora vamos a definir una frecuencia umbral por la que cortar el espectro, es decir, solo nos quedaremos con aquellas frecuencias que esten por debajo de este valor para el archivo de audio comprimido.
##### Con este fin, definimos el parámetro epsilon que representa la parte de la energía del espectro que NO conservaremos (la integral con respecto a la frecuencia).

In [ ]:
# Definimos diferentes epsilons: la parte de
# la energia del espectro que NO conservamos.
eps = [1e-5, .02, .041, .063, .086, .101, .123]

# Jugamos con los valores de epsilon (ID VARIANDO ESTE VALOR Y MIRAD LA GRÁFICA).
eps = eps[2]
print(f'Epsilon: {eps}')

# Calculamos el valor de corte para esta energia.
thr_spec_energy = (1 - eps) * np.sum(abs_ch_Fourier)
print(f'Valor de corte para la energia del espectro: {thr_spec_energy}')

# Integral de la frecuencia --> energia del espectro.
spec_energy = np.cumsum(abs_ch_Fourier)

# Mascara (array booleano) que compara el
# valor de corte con la energia del espectro.
frequencies_to_remove = thr_spec_energy < spec_energy  
print(f'Mascara: {frequencies_to_remove}')

# La frecuencia f0 por la que cortamos el espectro.
f0 = (len(frequencies_to_remove) - np.sum(frequencies_to_remove)) * (Fs/2) / (n//2)
print(f'Frecuencia de corte f0 (Hz): {int(f0)}')

# Graficamos.
plt.axvline(f0, color='r')
plt.plot(np.linspace(0, Fs/2, n//2), abs_ch_Fourier)
plt.ylabel('Amplitud')
plt.xlabel('$f$ (Hz)')
plt.show()

##### He seleccionado un valor de epsilon = 0.041. Esto significa que se descarta aproximadamente el 4.1% de la energía total del espectro y se conserva el 95.9% restante. He optado por este valor porque logra eliminar componentes de muy baja energía—posiblemente ruido—mientras se mantienen intactas las características principales de la señal, obteniendo así un espectro más limpio y representativo del audio.

## Comprensión del archivo
### Reducción de la resolución de muestreo (downsampling)
##### Para reducir el tamaño del archivo de audio, lo que vamos a hacer es aplicar downsampling.

In [ ]:
# Calculamos el factor D de downsampling.
D = int(Fs / f0)
print(f'Factor de downsampling: {D}')

# Obtenemos los nuevos datos (slicing with stride).
new_data = new_data_mono[::D]

# Definimos el nombre del audio comprimido generado.
wav_compressed_file = "the_last_of_us_compressed.wav"

# Escribimos los datos a un archivo de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path, wav_compressed_file),
    rate=int(Fs/D),
    data=new_data
)

# Cargamos el nuevo archivo.
new_sample_rate, new_audio_data = wavfile.read(filename=os.path.join(audio_output_path, wav_compressed_file))

##### Vamos a escucharlo

In [ ]:
IPython.display.Audio(new_audio_data, rate=new_sample_rate)

## Espectograma de ambas ondas

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

Pxx, freqs, bins, im = ax[0].specgram(new_data_mono, NFFT=1024, Fs=sample_rate, noverlap=512)
ax[0].set_title('Espectograma del audio original')
ax[0].set_ylabel('Frecuencia (Hz)')
ax[0].grid(True)

Pxx, freqs, bins, im = ax[1].specgram(new_audio_data, NFFT=1024, Fs=new_sample_rate, noverlap=512)
ax[1].set_title('Espectrograma del audio reducido/comprimido')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Frecuencia (Hz)')
ax[1].grid(True)

plt.tight_layout()
plt.show()

##### En el espectrograma original, se ve un rango amplio de frecuencias hasta la parte alta (zonas amarillas y verdes más arriba). En cambio, en el espectrograma comprimido, las frecuencias más altas casi desaparecen, porque hemos recortado (o reducido) la señal a una banda de frecuencias menor. Esto hace que el archivo sea más pequeño, pero perdemos parte de los sonidos agudos, y se nota menos “brillo” en la música.

## Tamaño de los archivos

In [ ]:
!ls -sh audio/_output/the_last_of_us_compressed.wav
!ls -sh audio/_output/the_last_of_us_reduced.wav

## Reproducción de los archivos de audio

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate) # Audio original

In [ ]:
IPython.display.Audio(new_audio_data, rate=new_sample_rate) # Audio comprimido